In [2]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

# Neo4j Import of GraphRAG Result Parquet files

This notebook imports the results of the GraphRAG indexing process into the Neo4j Graph database for further processing, analysis or visualization. 

You can also build your own GenAI applications using Neo4j and a number of RAG strategies with LangChain, LlamaIndex, Haystack, and many other frameworks.
See: https://neo4j.com/labs/genai-ecosystem

Here is what the end result looks like:

![](https://dev.assets.neo4j.com/wp-content/uploads/graphrag-neo4j-visualization.png)

## How does it work?

The notebook loads the parquet files from the `output` folder of your indexing process and loads them into Pandas dataframes.
It then uses a batching approach to send a slice of the data into Neo4j to create nodes and relationships and add relevant properties. The id-arrays on most entities are turned into relationships. 

All operations use MERGE, so they are idempotent, and you can run the script multiple times.

If you need to clean out the database, you can run the following statement

```cypher
MATCH (n)
CALL { WITH n DETACH DELETE n } IN TRANSACTIONS OF 25000 ROWS;
```

In [3]:
import os

GRAPHRAG_FOLDER = "C:\Code\III_2024_golf\\ragtest\output"

### Depedendencies

We only need Pandas and the neo4j Python driver with the rust extension for faster network transport.

In [4]:
import time

import pandas as pd
from neo4j import GraphDatabase

## Neo4j Installation

You can create a free instance of Neo4j [online](https://console.neo4j.io). You get a credentials file that you can use for the connection credentials. You can also get an instance in any of the cloud marketplaces.

If you want to install Neo4j locally either use [Neo4j Desktop](https://neo4j.com/download) or 
the official Docker image: `docker run -e NEO4J_AUTH=neo4j/password -p 7687:7687 -p 7474:7474 neo4j` 

In [5]:
NEO4J_URI="neo4j+s://2e42fd1b.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="aj9-m9QwIdnaSmmMl7oqeoIj_xZ6vCNjN5Jv-mr8I_E"
NEO4J_DATABASE="neo4j"
# Create a Neo4j driver
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

## Batched Import

The batched import function takes a Cypher insert statement (needs to use the variable `value` for the row) and a dataframe to import.
It will send by default 1k rows at a time as query parameter to the database to be inserted.

In [6]:
def batched_import(statement, df, batch_size=1000):
    """
    Import a dataframe into Neo4j using a batched approach.

    Parameters: statement is the Cypher query to execute, df is the dataframe to import, and batch_size is the number of rows to import in each batch.
    """
    total = len(df)
    start_s = time.time()
    for start in range(0, total, batch_size):
        batch = df.iloc[start : min(start + batch_size, total)]
        result = driver.execute_query(
            "UNWIND $rows AS value " + statement,
            rows=batch.to_dict("records"),
            database_=NEO4J_DATABASE,
        )
        print(result.summary.counters)
    print(f"{total} rows in {time.time() - start_s} s.")
    return total

## Indexes and Constraints

Indexes in Neo4j are only used to find the starting points for graph queries, e.g. quickly finding two nodes to connect.
Constraints exist to avoid duplicates, we create them mostly on id's of Entity types.

We use some Types as markers with two underscores before and after to distinguish them from the actual entity types.

The default relationship type here is `RELATED` but we could also infer a real relationship-type from the description or the types of the start and end-nodes.

* `__Entity__`
* `__Document__`
* `__Chunk__`
* `__Community__`
* `__Covariate__`

In [7]:
# create constraints, idempotent operation

statements = [
    "\ncreate constraint chunk_id if not exists for (c:__Chunk__) require c.id is unique",
    "\ncreate constraint document_id if not exists for (d:__Document__) require d.id is unique",
    "\ncreate constraint entity_id if not exists for (c:__Community__) require c.community is unique",
    "\ncreate constraint entity_id if not exists for (e:__Entity__) require e.id is unique",
    "\ncreate constraint entity_title if not exists for (e:__Entity__) require e.name is unique",
    "\ncreate constraint entity_title if not exists for (e:__Covariate__) require e.title is unique",
    "\ncreate constraint related_id if not exists for ()-[rel:RELATED]->() require rel.id is unique",
    "\n",
]

for statement in statements:
    if len((statement or "").strip()) > 0:
        print(statement)
        driver.execute_query(statement)


create constraint chunk_id if not exists for (c:__Chunk__) require c.id is unique

create constraint document_id if not exists for (d:__Document__) require d.id is unique

create constraint entity_id if not exists for (c:__Community__) require c.community is unique

create constraint entity_id if not exists for (e:__Entity__) require e.id is unique

create constraint entity_title if not exists for (e:__Entity__) require e.name is unique

create constraint entity_title if not exists for (e:__Covariate__) require e.title is unique

create constraint related_id if not exists for ()-[rel:RELATED]->() require rel.id is unique


## Import Process

### Importing the Documents

We're loading the parquet file for the documents and create nodes with their ids and add the title property.
We don't need to store text_unit_ids as we can create the relationships and the text content is also contained in the chunks.

In [8]:
doc_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_documents.parquet", columns=["id", "title"]
)
doc_df.head(2)

,id,title
0,79a497eed050a67abeac53133083370776d5896e1f0e60...,現今高爾夫球桿基本概念.txt
1,bba9ca74289df8e717153c49b987e9cd05361bdf376440...,綜評高爾夫下揮桿之腕部釋放技巧.txt


In [9]:
# Import documents
statement = """
MERGE (d:__Document__ {id:value.id})
SET d += value {.title}
"""

batched_import(statement, doc_df)

{'_contains_updates': True, 'properties_set': 7}
7 rows in 0.17824578285217285 s.


7

### Loading Text Units

We load the text units, create a node per id and set the text and number of tokens.
Then we connect them to the documents that we created before.

In [10]:
text_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_text_units.parquet",
    columns=["id", "text", "n_tokens", "document_ids"],
)
text_df.head(2)

,id,text,n_tokens,document_ids
0,23f9f55a3d55f4d9c08729314d06b8454c3cb06b3e2277...,高爾夫球桿桿身的相關研究\n\ndoi:10.29706/GS.201208.0007\n大...,1200,[0fc45284410a8448f5e5b3553f77b2ed518df093fd089...
1,bbec972c236d5442cf6c1d6ad96c4a9110148e6cbe3a4b...,業餘的高爾夫愛好者在合適的球具、適當的練習下，即能夠在很\n短的時間，就具備不錯的揮桿距離與...,1200,[0fc45284410a8448f5e5b3553f77b2ed518df093fd089...


In [11]:
statement = """
MERGE (c:__Chunk__ {id:value.id})
SET c += value {.text, .n_tokens}
WITH c, value
UNWIND value.document_ids AS document
MATCH (d:__Document__ {id:document})
MERGE (c)-[:PART_OF]->(d)
"""

batched_import(statement, text_df)

{'_contains_updates': True, 'properties_set': 116}
58 rows in 0.395430326461792 s.


58

### Loading Nodes

For the nodes we store id, name, description, embedding (if available), human readable id.

In [12]:
entity_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_entities.parquet",
    columns=[
        "id",  
        "human_readable_id",    
        "title",
        "type",
        "description",
        "text_unit_ids",
    ],
)
entity_df.head(2)

,id,human_readable_id,title,type,description,text_unit_ids
0,bf2440ec-3d12-498c-9ed3-ac7576270785,0,王順正,PERSON,王順正 is an author affiliated with National Chun...,[23f9f55a3d55f4d9c08729314d06b8454c3cb06b3e227...
1,f4907ce6-5165-487a-bfda-cffcb579ce2b,1,林玉瓊,PERSON,林玉瓊 is an author affiliated with Wu Feng Unive...,[23f9f55a3d55f4d9c08729314d06b8454c3cb06b3e227...


In [13]:
entity_statement = """
MERGE (e:__Entity__ {id:value.id})
SET e += value {.human_readable_id, .description, title:replace(value.title,'"','')}
WITH e, value
CALL apoc.create.addLabels(e, case when coalesce(value.type,"") = "" then [] else [apoc.text.upperCamelCase(replace(value.type,'"',''))] end) yield node
UNWIND value.text_unit_ids AS text_unit
MATCH (c:__Chunk__ {id:text_unit})
MERGE (c)-[:HAS_ENTITY]->(e)
"""

batched_import(entity_statement, entity_df)

{'_contains_updates': True, 'properties_set': 1887}
629 rows in 0.9972748756408691 s.


629

### Import Relationships

For the relationships we find the source and target node by name, using the base `__Entity__` type.
After creating the `RELATED` relationships, we set the description as attribute.

In [14]:
rel_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_relationships.parquet",
    columns=[
        "id",
        "human_readable_id",
        "source",
        "target",
        "description",
        "weight",
        "combined_degree",
        "text_unit_ids"
    ],
)
rel_df.head(2)

,id,human_readable_id,source,target,description,weight,combined_degree,text_unit_ids
0,dd3e85e6-fcab-4d49-86f8-fa2fc2b3545f,0,王順正,高爾夫球桿,王順正 is an author who conducted research on gol...,8.0,45,[23f9f55a3d55f4d9c08729314d06b8454c3cb06b3e227...
1,3164e95b-f59f-49b7-b85c-1cb6a0ccb889,1,林玉瓊,高爾夫球桿,林玉瓊 is an author who conducted research on gol...,8.0,45,[23f9f55a3d55f4d9c08729314d06b8454c3cb06b3e227...


In [15]:
rel_statement = """
    MATCH (source:__Entity__ {name:replace(value.source,'"','')})
    MATCH (target:__Entity__ {name:replace(value.target,'"','')})
    // not necessary to merge on id as there is only one relationship per pair
    MERGE (source)-[rel:RELATED {id: value.id}]->(target)
    SET rel += value {.rank, .weight, .human_readable_id, .description, .text_unit_ids}
    RETURN count(*) as createdRels
"""

batched_import(rel_statement, rel_df)

{}
818 rows in 0.3545053005218506 s.


818

### Importing Communities

For communities we import their id, title, level.
We connect the `__Community__` nodes to the start and end nodes of the relationships they refer to.

Connecting them to the chunks they orignate from is optional, as the entites are already connected to the chunks.

In [16]:
community_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_communities.parquet",
    columns=["id", "level", "title", "text_unit_ids", "relationship_ids"],
)

community_df.head(2)

,id,level,title,text_unit_ids,relationship_ids
0,19e5deee-4930-4811-b758-b7bcead41273,0,Community 0,[0b8bb9591af8ed28801e86e8a5bc1de89e027c48434fd...,"[0188c1f0-e2e6-42d6-a140-544efb447bf2, 0378d07..."
1,ac5e9bda-c3a5-428a-930d-7af7c99f11d3,0,Community 1,[63460779082864c6d13982183b698189691ca36e76117...,"[0ca09d3e-b942-4dbe-91a0-806aab6c20ad, 104a829..."


In [19]:
statement = """
MERGE (c:__Community__ {community:value.id})
SET c += value {.level, .title}
/*
UNWIND value.text_unit_ids as text_unit_id
MATCH (t:__Chunk__ {id:text_unit_id})
MERGE (c)-[:HAS_CHUNK]->(t)
WITH distinct c, value
*/
WITH *
UNWIND value.relationship_ids as rel_id
MATCH (start:__Entity__)-[:RELATED {id:rel_id}]->(end:__Entity__)
MERGE (start)-[:IN_COMMUNITY]->(c)
MERGE (end)-[:IN_COMMUNITY]->(c)
RETURn count(distinct c) as createdCommunities
"""

batched_import(statement, community_df)

{'_contains_updates': True, 'properties_set': 226}
113 rows in 0.8893873691558838 s.


113

### Importing Community Reports

Fo the community reports we create nodes for each communitiy set the id, community, level, title, summary, rank, and rank_explanation and connect them to the entities they are about.
For the findings we create the findings in context of the communities.

In [182]:
community_report_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_community_reports.parquet",
    columns=[
        "id",
        "community",
        "level",
        "title",
        "summary",
        "findings",
        "rank",
        "rank_explanation",
        "full_content",
    ],
)
community_report_df.head(2)

,id,community,level,title,summary,findings,rank,rank_explanation,full_content
0,7f94ed38c3f945e3ad1ad19fa548a3ba,83,2,力量与速度结合原则与Wenger与Bell的研究,该社区围绕力量与速度结合原则及其理论基础展开，Wenger与Bell的研究为这一原则提供了重...,[{'explanation': '力量与速度结合原则是高尔夫挥杆中的关键概念，运动员需要合...,4.0,影响严重性评分为中等，主要由于力量与速度结合原则在高尔夫运动中的应用可能影响运动员的表现。,# 力量与速度结合原则与Wenger与Bell的研究\n\n该社区围绕力量与速度结合原则及其...
1,7768076af1e14a3e8a887a481ad52c3a,84,2,Golf Swing Mechanics and Muscle Dynamics,This community focuses on the key muscle group...,[{'explanation': 'The community identifies sev...,7.5,The impact severity rating is high due to the ...,# Golf Swing Mechanics and Muscle Dynamics\n\n...


In [183]:
# Import communities
community_statement = """
MERGE (c:__Community__ {community:value.community})
SET c += value {.level, .title, .rank, .rank_explanation, .full_content, .summary}
WITH c, value
UNWIND range(0, size(value.findings)-1) AS finding_idx
WITH c, value, finding_idx, value.findings[finding_idx] as finding
MERGE (c)-[:HAS_FINDING]->(f:Finding {id:finding_idx})
SET f += finding
"""
batched_import(community_statement, community_report_df)

{'_contains_updates': True, 'labels_added': 657, 'relationships_created': 544, 'nodes_created': 657, 'properties_set': 2423}
113 rows in 0.4137256145477295 s.


113

### Importing Covariates

Covariates are for instance claims on entities, we connect them to the chunks where they originate from.

In [184]:

cov_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_covariates.parquet",
    columns=["id","text_unit_id"],
)
cov_df
# Subject id do not match entity ids

,id,text_unit_id
0,ea07d202-a5e1-46c1-b0b1-1a4738c5b5a9,48247d3af85c659be6862abaeba68005bc0deb1bd78fb1...
1,af3f53b6-d5e9-440b-92cd-de6a29b161dc,18672b338b62eebb11280822128bcf852cf27e6cf4fb32...
2,6f277700-b111-4359-89a8-5f99e5cff1db,18672b338b62eebb11280822128bcf852cf27e6cf4fb32...
3,bebee862-4d7c-496d-8f74-14510bf2f3ae,18672b338b62eebb11280822128bcf852cf27e6cf4fb32...
4,3063ce62-4ad8-49c6-9ecc-5d6204a7fb4a,18672b338b62eebb11280822128bcf852cf27e6cf4fb32...
5,d9d5fcc2-60bb-4534-8b4b-3ffaf1943158,8a8630fef2d71aa677e81a096116ac8be1a1e557921017...
6,a3cf088f-9177-4acc-b666-8b28c5109f33,707ec6b02b34ab5b70c8edde36cfa346dc02a7325970ef...
7,7cfcf005-eb80-4734-bc1c-8f6d906fa400,db621b0d020fa7a74d3d331770459c4a8d8dce6abd59d5...
8,d2a6010c-4b7d-489f-8198-d03452d6da04,bfbb21f7a4a1ece03c36fe70e4a6b7aa3c6c60b340501a...
9,e88cf8c9-aab4-4e8c-bc45-750b91edf5a1,466a48ca12f31c2170e68b43e1e66a5e5851c62df7582c...


In [185]:
# Import covariates
cov_statement = """
MERGE (c:__Covariate__ {id:value.id})
SET c += apoc.map.clean(value, ["text_unit_id", "document_ids", "n_tokens"], [NULL, ""])
WITH c, value
MATCH (ch:__Chunk__ {id: value.text_unit_id})
MERGE (ch)-[:HAS_COVARIATE]->(c)
"""
batched_import(cov_statement, cov_df)

{'_contains_updates': True, 'labels_added': 14, 'nodes_created': 14, 'properties_set': 28}
14 rows in 0.16161847114562988 s.


14

### Visualize your data

You can now [Open] Neo4j on Aura, you need to log in with either SSO or your credentials.

Or open https://workspace-preview.neo4j.io and connect to your local instance, remember the URI is `neo4j://localhost` and `neo4j` as username and `password` as password.

In "Explore" you can explore by using visual graph patterns and then explore and expand further.

In "Query", you can open the left sidebar and explore by clicking on the nodes and relationships.
You can also use the co-pilot to generate Cypher queries for your, here are some examples.

#### Show a few `__Entity__` nodes and their relationships (Entity Graph)

```cypher
MATCH path = (:__Entity__)-[:RELATED]->(:__Entity__)
RETURN path LIMIT 200
```

#### Show the Chunks and the Document (Lexical Graph)

```cypher
MATCH (d:__Document__) WITH d LIMIT 1
MATCH path = (d)<-[:PART_OF]-(c:__Chunk__)
RETURN path LIMIT 100
```

####  Show a Community and it's Entities

```cypher
MATCH (c:__Community__) WITH c LIMIT 1
MATCH path = (c)<-[:IN_COMMUNITY]-()-[:RELATED]-(:__Entity__)
RETURN path LIMIT 100
```

#### Show everything

```cypher
MATCH (d:__Document__) WITH d LIMIT 1
MATCH path = (d)<-[:PART_OF]-(:__Chunk__)-[:HAS_ENTIY]->()-[:RELATED]-()-[:IN_COMMUNITY]->()
RETURN path LIMIT 250
```

We showed the visualization of this last query at the beginning.

If you have questions, feel free to reach out in the GraphRAG discord server: 
https://discord.gg/graphrag